In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import random
import pickle
import argparse
from copy import deepcopy
from IPython.display import clear_output
import time


import ipywidgets as widgets
import functools

from l_system import *
from utils import *

In [2]:
# create the l-system object
l_system=L_system()
# initiate the l-system's vocabulary
l_system.update_vocabulary({'X','F','-','+','[',']'})
# add the l-system rules
#l_system.add_rule('X', 'F+[[X]-X]-F[-FX]+X',verbose=True)
l_system.add_rule('X', 'F[+X]-F',verbose=True)
l_system.add_rule('F', 'FF',verbose=True)
#create the interpreter of the l-system - interpreting it as a rule for tree grow
tree_interpreter=Tree_interpreter(plus_angle=np.pi/8,minus_angle=-np.pi/6,randomize_angle=False)
n_iter=5

In [3]:
#image resolution
IMG_WIDTH = 400
IMG_HEIGHT = IMG_WIDTH
#number of total tree created


def get_offspring(ls,ti,n_iter):
    images=[]#here we collect the tree images
    ls_offspring=[ls]#here we collect the l-system objects of the offspring
    ti_offspring=[ti]#here we collect the tree interpreter objects of the offspring
    n_iter_offspring=[n_iter]#here we collect the recursive depth of the offspring
    w,_=ls.evolution('X',n_iter)#this is the recursive word creation
    segments=ti.render(w)#the word is rendered to a tree
    im=draw_tree(segments,im_size=IMG_WIDTH)#the tree is drawn to an actual image
    #we convert the PIL image to a png image
    im.save('images/tree.png')
    file = open("images/tree.png", "rb")
    png_image = file.read()
    images.append(png_image)
    print('generate offspring...')
    #this is the mutation loop for the offspring
    for i in range(n_trees-1):
        ls_copy=deepcopy(ls)
        ti_copy=deepcopy(ti)
        #apply mutations
        ls_copy.point_mutation('X',0.04,immune={'[',']'})
        ls_copy.perm_mutation('X',0.05)
        ls_copy.loss_mutation('X',0.05,immune={'[',']'})
        ls_copy.add_mutation('X',0.03,words={'[X]','[F]'},immune={'[',']'})
        ti_copy.angle_mutation()
        ls_offspring.append(ls_copy)
        ti_offspring.append(ti_copy)
        r=np.random.rand()
        if r<0.1:
            n=min(9,n_iter+1)
        elif r>0.9:
            n=max(2,n_iter-1)
        else:
            n=n_iter
        n_iter_offspring.append(n)
        w,_=ls_copy.evolution('X',n)#create the recursive word off the mutated tree
        im=draw_tree(ti_copy.render(w),im_size=IMG_WIDTH)#interprete it as a tree and draw it
        #convert the PIL image to png formate
        im.save('images/tree.png')
        file = open("images/tree.png", "rb")
        png_image = file.read()
        images.append(png_image)
    return images,ls_offspring,ti_offspring,n_iter_offspring


In [14]:
#number of colums and rows shown
COLS = 8
ROWS = 8
n_trees=COLS*ROWS
#the function on_click defines what happens if we select a parental tree
def on_click(b):
    global ls_offspring,ti_offspring,n_iter_offspring,IMAGES
    clear_output()
    number_string=b.description.split(' ')[0]
    tree_number=int(number_string)
    selected_img=widgets.Image(
        value=IMAGES[tree_number],
        format='png',
        width=500,
        height=500,
        )
    print('selected tree: '+str(tree_number))
    print('gene code: '+str(ls_offspring[tree_number].rule['X']))
    print('age: '+str(n_iter_offspring[tree_number]))
    display(selected_img)
    start_time = time.time()
    IMAGES,ls_offspring,ti_offspring,n_iter_offspring=get_offspring(ls_offspring[tree_number],ti_offspring[tree_number],n_iter_offspring[tree_number])
    duration=time.time()-start_time
    show_offsprings(IMAGES,ROWS,COLS,IMG_WIDTH=IMG_WIDTH,IMG_HEIGHT=IMG_HEIGHT,duration=duration)


#this function actually arranges the offspring in a nice table with bottoms to click
def show_offsprings(IMAGES,ROWS,COLS,IMG_WIDTH=200,IMG_HEIGHT=200,duration=0):
    rows = []
    start_time=time.time()
    for row in range(ROWS):
        cols = []
        for col in range(COLS):
            index = row * COLS + col
            image = widgets.Image(
                value=IMAGES[index], width=IMG_WIDTH, height=IMG_HEIGHT
            )
            if index==0:
                add_string=' ancestor'
            else:
                add_string=' '
            button = widgets.Button(description=str(index)+add_string)
            button.on_click(on_click)
            box = widgets.VBox([image, button])
            cols.append(box)
        # Create a horizontal layout box, grouping all the columns together
        rows.append(widgets.HBox(cols))
    # Create a vertical layout box, grouping all the rows together
    result = widgets.VBox(rows)
    time.sleep(max(0,5-(time.time()-start_time)-duration))
    clear_output()
    display(result)
    return result
    
IMAGES,ls_offspring,ti_offspring,n_iter_offspring=get_offspring(l_system,tree_interpreter,n_iter)

generate offspring...


In [13]:
_=show_offsprings(IMAGES,ROWS,COLS,IMG_WIDTH=IMG_WIDTH,IMG_HEIGHT=IMG_HEIGHT)

selected tree: 26
gene code: F+[X]-F
age: 5


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01\x90\x08\x02\x00\x00\x00\x0f\xdd\…

generate offspring...
